In [ ]:
%install '.package(url: "https://github.com/mxcl/Path.swift", from: "0.16.1")' Path
%install '.package(url: "https://github.com/JustHTTP/Just", from: "0.7.1")' Just

Installing packages:
	.package(url: "https://github.com/mxcl/Path.swift", from: "0.16.1")
		Path
	.package(url: "https://github.com/JustHTTP/Just", from: "0.7.1")
		Just
Working in: /tmp/tmp9o9ywnpp
Fetching https://github.com/mxcl/Path.swift
Fetching https://github.com/JustHTTP/Just
Completed resolution in 0.97s
Cloning https://github.com/mxcl/Path.swift
Resolving https://github.com/mxcl/Path.swift at 0.16.2
Cloning https://github.com/JustHTTP/Just
Resolving https://github.com/JustHTTP/Just at 0.7.1
Compile Swift Module 'Just' (1 sources)
Compile Swift Module 'Path' (9 sources)
Compile Swift Module 'jupyterInstalledPackages' (1 sources)
Linking ./.build/x86_64-unknown-linux/debug/libjupyterInstalledPackages.so
Installation complete!

## Getting the MNIST dataset

In [ ]:
// export
import Foundation
import Path
import Just

In [ ]:
//export
public func shell_cmd(_ launchPath: String, _ arguments: [String]) -> String?
{
    let task = Process()
    task.executableURL = URL.init(fileURLWithPath:launchPath)
    task.arguments = arguments

    let pipe = Pipe()
    task.standardOutput = pipe
    do {try task.run()} catch {print("Unexpected error: \(error).")}

    let data = pipe.fileHandleForReading.readDataToEndOfFile()
    let output = String(data: data, encoding: String.Encoding.utf8)

    return output
}

In [ ]:
if let res = shell_cmd("/bin/ls", ["-lh"]){print(res)}

total 9.6M
-rw-rw-r-- 1 ubuntu ubuntu  11K Mar 27 13:34 00_load_data.ipynb
-rw-rw-r-- 1 ubuntu ubuntu  13K Mar 27 13:19 01_matmul.ipynb
drwxrwxr-x 3 ubuntu ubuntu 4.0K Mar 26 19:58 FastaiNotebooks
drwxrwxr-x 3 ubuntu ubuntu 4.0K Mar 26 19:58 Notebook2Script
-rw-rw-r-- 1 ubuntu ubuntu 9.3K Mar 26 15:03 Notebook2Script.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 9.5M Mar 26 14:43 train-images-idx3-ubyte.gz



In [ ]:
//export
public func download_file(_ url: String, dest: String?=nil, force: Bool=false){
    let dest_name = (dest ?? (Path.cwd/url.split(separator: "/").last!).string)
    let url_dest = URL.init(fileURLWithPath: (dest ?? (Path.cwd/url.split(separator: "/").last!).string))
    if (force || !Path(dest_name)!.exists){
        print("Downloading \(url)...")
        if let cts = Just.get(url).content{
            do    {try cts.write(to: URL.init(fileURLWithPath:dest_name))}
            catch {print("Can't write to \(url_dest).\n\(error)")}
        } else {print("Can't reach \(url)")}
    }
}

In [ ]:
download_file("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz")

In [ ]:
//export
import TensorFlow

In [ ]:
//export
protocol ConvertableFromByte {
    init(_ d:UInt8)
}

In [ ]:
//export
extension Float : ConvertableFromByte{}
extension Int32 : ConvertableFromByte{}

In [ ]:
//export
func get_data<T:ConvertableFromByte & TensorFlowScalar>(_ fn:String, _ skip:Int) -> Tensor<T> {
    let data = try! Data.init(contentsOf: URL.init(fileURLWithPath: fn)).dropFirst(skip)
    return Tensor(data.map(T.init))
}

In [ ]:
// export
public struct MnistDataset{
    let base_url = "http://yann.lecun.com/exdb/mnist/"
    let trn_imgs = "train-images-idx3-ubyte"
    let trn_lbls = "train-labels-idx1-ubyte"
    let val_imgs = "t10k-images-idx3-ubyte"
    let val_lbls = "t10k-labels-idx1-ubyte" 
    
    var path = Path.cwd
    
    public init(path: Path){
        self.path = path
        if !path.exists {try! path.mkdir(.p)}
        let data_files = [path/trn_imgs, path/trn_lbls, path/val_imgs, path/val_lbls]
        for file in data_files{
            if !file.exists {
                let fname = file.basename()
                download_file("\(base_url)\(fname).gz", dest:(path/"\(fname).gz").string)
                _ = shell_cmd("/bin/gunzip", ["-fq", (path/"\(fname).gz").string])
            }
        }
    }
    
    func get_data<T:ConvertableFromByte & TensorFlowScalar>(_ fn:String, _ skip:Int) -> Tensor<T> {
        let data = try! Data.init(contentsOf: URL.init(fileURLWithPath: fn)).dropFirst(skip)
        return Tensor(data.map(T.init))
    }
    
    public var xTrain: Tensor<Float> {return get_data((path/trn_imgs).string, 16)/255.0}
    public var yTrain: Tensor<Int32> {return get_data((path/trn_lbls).string, 8)}
    public var xValid: Tensor<Float> {return get_data((path/val_imgs).string, 16)/255.0}
    public var yValid: Tensor<Int32> {return get_data((path/val_lbls).string, 8)}
}

In [ ]:
let mnist = MnistDataset(path: Path.home/".fastai"/"data"/"mnist_tst")

## Timing

In [ ]:
//export 
import Dispatch
public func time(_ function: () -> ()) {
    let start = DispatchTime.now()
    function()
    let end = DispatchTime.now()
    let nanoseconds = Double(end.uptimeNanoseconds - start.uptimeNanoseconds)
    let milliseconds = nanoseconds / 1e6
    print("\(milliseconds) ms")
}

In [ ]:
time {var trn_imgs = mnist.xTrain}

4656.767389 ms


In [ ]:
//export 
public func time(repeating: Int, _ function: () -> ()) {
    var times:[Double] = []
    for _ in 1...repeating{
        let start = DispatchTime.now()
        function()
        let end = DispatchTime.now()
        let nanoseconds = Double(end.uptimeNanoseconds - start.uptimeNanoseconds)
        let milliseconds = nanoseconds / 1e6
        times.append(milliseconds)
    }
    print("\(times.reduce(0.0, +)/Double(times.count)) ms")
}

In [ ]:
time(repeating:10) {var trn_imgs = mnist.xTrain}

1276.9919814999998 ms


## Export

In [ ]:
// export
func notebook2script(fname: String){
    let url_fname = URL.init(fileURLWithPath: fname)
    let last = fname.lastPathComponent
    let out_fname = (url_fname.deletingLastPathComponent().appendingPathComponent("FastaiNotebooks", isDirectory: true)
                     .appendingPathComponent("Sources", isDirectory: true)
                     .appendingPathComponent("FastaiNotebooks", isDirectory: true).appendingPathComponent(last)
                     .deletingPathExtension().appendingPathExtension("swift"))
    do{
        let data = try Data.init(contentsOf: url_fname)
        let json_data = try! JSONSerialization.jsonObject(with: data, options: .allowFragments) as! [String: Any]
        let cells = json_data["cells"] as! [[String:Any]]
        var module = """
/*
THIS FILE WAS AUTOGENERATED! DO NOT EDIT!
file to edit: \(fname.lastPathComponent)

*/
        
"""
        for cell in cells{
            if let source = cell["source"] as? [String]{
                if source.isEmpty {continue}
                if source[0].range(of: #"^\s*//\s*export\s*$"#, options: .regularExpression) != nil{
                    module.append("\n" + source[1...].joined() + "\n")
                }
            }
        }
        try? module.write(to: out_fname, atomically: false, encoding: .utf8)
    } catch {print("Can't read the content of \(fname)")}
}

In [ ]:
// export
public func export_notebooks(_ path: Path){
    for entry in try! path.ls(){
        if entry.kind == Entry.Kind.file{
            if entry.path.basename().range(of: #"^\d*_.*ipynb$"#, options: .regularExpression) != nil { 
                print("Converting \(entry.path.basename())")
                notebook2script(fname: entry.path.basename())
            }
        }
    }
}

In [ ]:
export_notebooks(Path.cwd)

Converting 00_load_data.ipynb
Converting 01_matmul.ipynb


In [ ]:
let files = Path.cwd.ls()

In [ ]:
print(files[1])

Entry(kind: Path.Entry.Kind.file, path: Path(/home/ubuntu/notebooks/swift/00_load_data.ipynb))


In [ ]:
files[3].path.basename().range(of: #"^\d*_.*ipynb$"#, options: .regularExpression)

nil


In [ ]:
files[1].path.basename().split(separator:"_")[0].

: 